# GLLVM Simulations

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from gllvm import GLLVM, PoissonGLM, BinomialGLM, GaussianGLM

# ----------------------------------------------------------
# 1. Generate synthetic data from the ground-truth model
# ----------------------------------------------------------

gllvm0 = GLLVM(latent_dim=1, output_dim=5)
gllvm0.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm0.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm0.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

num_samples = 10_000
z0 = gllvm0.sample_z(num_samples)
y0 = gllvm0.sample(z=z0)

# ----------------------------------------------------------
# 2. Build a fresh model that will be trained with VI
# ----------------------------------------------------------

gllvm = GLLVM(latent_dim=1, output_dim=5)
gllvm.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

# ----------------------------------------------------------
# 3. Build the encoder q(z|y)
# ----------------------------------------------------------

class Encoder(nn.Module):
    def __init__(self, input_dim=5, latent_dim=1, hidden=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )
        self.mean = nn.Linear(hidden, latent_dim)
        self.logvar = nn.Linear(hidden, latent_dim)

    def forward(self, y):
        h = self.net(y)
        mu = self.mean(h)
        logvar = self.logvar(h)
        return mu, logvar

encoder = Encoder(input_dim=5, latent_dim=1)
optimizer = optim.Adam(list(gllvm.parameters()) + list(encoder.parameters()), lr=1e-3)

# ----------------------------------------------------------
# 4. VI training loop
# ----------------------------------------------------------

batch_size = 256
num_epochs = 1000

dataset = y0
n = len(dataset)

for epoch in range(num_epochs):
    perm = torch.randperm(n)

    total_elbo = 0.0
    for i in range(0, n, batch_size):
        idx = perm[i:i+batch_size]
        y = dataset[idx]

        # Encode q(z|y)
        mu, logvar = encoder(y)
        std = torch.exp(0.5 * logvar)

        # Reparameterization
        eps = torch.randn_like(std)
        z = mu + eps * std

        # Decoder log-likelihood
        logpy_z = gllvm.log_prob(y, z=z).sum(dim=-1)  # sum over response dims

        # KL(q||p)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)

        elbo = (logpy_z - kl).mean()
        loss = -elbo

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_elbo += elbo.item()

    print(f"Epoch {epoch+1}: ELBO={total_elbo:.2f}")


Epoch 1: ELBO=-134.54
Epoch 2: ELBO=-94.56
Epoch 3: ELBO=-77.88
Epoch 4: ELBO=-55380025.44
Epoch 5: ELBO=-1315.35
Epoch 6: ELBO=-62.96
Epoch 7: ELBO=-69.32
Epoch 8: ELBO=-73.52
Epoch 9: ELBO=-81.13
Epoch 10: ELBO=-82.79
Epoch 11: ELBO=-88.58
Epoch 12: ELBO=-87.41
Epoch 13: ELBO=-85.71
Epoch 14: ELBO=-84.01
Epoch 15: ELBO=-82.53
Epoch 16: ELBO=-80.73
Epoch 17: ELBO=-79.09
Epoch 18: ELBO=-75.05
Epoch 19: ELBO=-67.99
Epoch 20: ELBO=-65.13
Epoch 21: ELBO=-55.65
Epoch 22: ELBO=-52.58
Epoch 23: ELBO=-53.02
Epoch 24: ELBO=-51.69
Epoch 25: ELBO=-50.21
Epoch 26: ELBO=-50.67
Epoch 27: ELBO=-49.97
Epoch 28: ELBO=-49.90
Epoch 29: ELBO=-59.22
Epoch 30: ELBO=-51.01
Epoch 31: ELBO=-53.43
Epoch 32: ELBO=-53.93
Epoch 33: ELBO=-52.65
Epoch 34: ELBO=-51.57
Epoch 35: ELBO=-50.43
Epoch 36: ELBO=-50.01
Epoch 37: ELBO=-49.12
Epoch 38: ELBO=-46.45
Epoch 39: ELBO=-46.72
Epoch 40: ELBO=-46.68
Epoch 41: ELBO=-46.61
Epoch 42: ELBO=-46.00
Epoch 43: ELBO=-45.23
Epoch 44: ELBO=-45.45
Epoch 45: ELBO=-45.82
Epoch 46: 

In [9]:
print("True W_z:\n", gllvm0.wz)
print("Estimated W_z:\n", gllvm.wz)

print("True bias:\n", gllvm0.bias)
print("Estimated bias:\n", gllvm.bias)

print("True scale:\n", gllvm0.scale)
print("Estimated scale:\n", gllvm.scale)


True W_z:
 Parameter containing:
tensor([[-0.9193,  1.5067, -1.4939,  0.1387, -0.9420]], requires_grad=True)
Estimated W_z:
 Parameter containing:
tensor([[-1.1198, -1.7769,  1.0145, -0.1205,  0.7829]], requires_grad=True)
True bias:
 Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
Estimated bias:
 Parameter containing:
tensor([-0.0239, -0.0143,  0.0706, -0.0099,  0.0776], requires_grad=True)
True scale:
 Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Estimated scale:
 Parameter containing:
tensor([1.0000, 1.0000, 1.0000, 1.0855, 1.6392], requires_grad=True)
